In [1]:
import csv
from datetime import datetime
from elasticsearch import Elasticsearch
import json


In [2]:
# settings
es = Elasticsearch([{'host': '127.0.0.1', 'port': 9292}])
indexname = "fi_news_dep"
importfile = "C:/Users/hitec_c/Documents/finnews_dep_wft.txt"


In [ ]:
# read docs use counter as additional id to start and stop pushing to elasticsearch
docs = []

# start and end rows are included in the dataset
start = 25000
end = 25000

with open(importfile, newline='', encoding="UTF-8") as f:
    reader = csv.reader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
    counter = 0
    for row in reader:
        if counter >= start:
            ############### parse
            josi = row[0]
            jos = josi.split("<>")
            #print("jos", jos)
            bimsi = row[1]
            bims = bimsi.split("<>")
            #print("bims", bims)
            source = row[2]
            #print("source", source)
            sentence = row[3]
            #print("sentence", sentence)
            time_slice = row[4]
            #print("time_slice", time_slice)
        
            ######### index in es
            doc = {
                'jos': jos,
                'bims': bims,
                'sentence': sentence,
                'source': source,
                'date': time_slice,
                'time_slice': time_slice
            }
            es.index(index=indexname, body=doc)
        
        ##### count and break
        if counter >=end:
            break
        counter += 1

In [3]:
# test query
jo = "että#SCONJ"
bim = "että#SCONJ@hyvä#ADJ#mark"
res = es.search(index=indexname, 
            body={"query": 
                {"bool":
                    {"must":
                        [
                        {"match": 
                            {"jos":  jo}
                        },
                        {"match": 
                            {"bims": bim}
                        }
                        ]
                    }
                }
            }
        )
#print(res)
ret_set = set()
for hit in res["hits"]["hits"]:
    text = hit["_source"]["date"][:10]+ " [" + str(hit["_source"]["time_slice"])+ "] : " + hit["_source"]["sentence"] + " [" + hit["_source"]["source"] + "] "
    ret_set.add(text)
    ret_list = list(ret_set)
    ret_list.sort()
print(ret_list)

['1999-2002 [1999-2002] : Että on hyvää . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Hyvin usein Sjödinit saavat myös kuulla , että olisi varmasti sekä pojille että vanhemmille parempi , jos pojat kuolisivat mahdollisimman pian . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Hyvä että menin . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Kun löytää toisesta vähän virheitä voi ainakin hetken ajatella että on itse parempi . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : On hyvä , että kirkon taholla on tartuttu aiheeseen . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Parempi , että jäät kotiin lepäämään . " [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Riittää että on kyllin hyvä , aivan tavallinen isä tai äiti . [https://www.kielipankki.fi/corpora/lehdet90ff/] ', '1999-2002 [1999-2002] : Se on niin hy